# Estimating Roof Profiles

Jakarta: How to we characterize roof profiles?
- this is useful for estimating POA from GHI for fixed tilt solar

Street angle - this becomes the azimuth angle
Every street yields two angles at 180 degrees from each other

Two kinds of street in jakarta can be considered - EW and NS

How are roof angles distributed?
- roofs are either perpendicular from the street, or parallel to it.

As for angles, roofs are always tilted, around 30 degrees

assume that roofs are 30 degrees from horizontal, and consider the cases where they have east-west and north-south orientations in equal proportion

In [ ]:
import pvlib
import json
import os
import math
from pprint import pprint
from datetime import datetime
from pytz import timezone
import statistics

import pandas as pd

In [ ]:
api_key = '9nbF8PWcvcjXeQALyMXmOfmefnOTb942xxHGvR0M'
full_name = "David Pratama Widjaja"
affiliation = ""
email = "widjaja.david@ymail.com"
reason = 'Research into Indonesian and ASEAN energy transition'
join_mailing_list = 'true'

jakarta_lat = -6.175389
jakarta_lon = 106.827139
jakarta_data_source = 'himawari'
jakarta_tz = timezone('Asia/Jakarta')

amsterdam_lat = 52.372778
amsterdam_lon = 4.893611
amsterdam_data_source = 'meteosat'
amsterdam_tz = timezone('Europe/Amsterdam')

surabaya_lat = -7.245833
surabaya_lon = 112.737778
surabaya_data_source = 'himawari'
surabaya_tz = timezone('Asia/Makassar')

vancouver_lat = 49.260833
vancouver_lon = -123.113889
vancouver_data_source = 'goes'
vancouver_tz = timezone('America/Vancouver')

toronto_lat = 43.741667
toronto_lon = -79.373333
toronto_data_source = 'goes'
toronto_tz = timezone('America/Toronto')

In [ ]:
def get_tmy_from_nsrdb(lat, lon, timezone, api_key, full_name, email, affiliation, data_source='himawari'):

    if data_source == "himawari":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/himawari-tmy-download.csv"
    elif data_source == "goes":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-GOES-tmy-v4-0-0-download.csv"
    elif data_source == "meteosat":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-msg-v1-0-0-tmy-download.csv"
    else:
        raise ValueError(f"Invalid data source {data_source}")

    key = f'{data_source}_tmy'
    if os.path.exists(key) is False:
        os.mkdir(key)

    filename = f"pvlib_experiments/{key}/{data_source}_tmy_{lat}_{lon}.csv"
    metadata_filename = f"pvlib_experiments/{key}/{data_source}_tmy_{lat}_{lon}.json"

    if os.path.exists(filename) is False:
        df, metadata = pvlib.iotools.get_nsrdb_psm4_tmy(
            lat,
            lon,
            api_key,
            email,
            year='tmy',
            time_step=60,
            leap_day=False,
            full_name=full_name,
            affiliation=affiliation,
            utc=True,
            map_variables=True,
            url=url,
            timeout=30
        )
    
        df.to_csv(filename)
        with open(metadata_filename, mode='w') as f:
            json.dump(metadata, f)

    df = pd.read_csv(filename, index_col=0)
    with open(metadata_filename, mode='r') as f:
        metadata = json.load(f)

    df["Year"] = 1990
    df["sitetime"] = df.apply(lambda x: datetime(int(x["Year"]), int(x["Month"]), int(x["Day"]), int(x["Hour"]), int(x["Minute"])), axis=1)
    df.index = pd.DatetimeIndex(df["sitetime"], tz="UTC").tz_convert(timezone)


    df = df[['temp_air', 'temp_dew', 'dhi', 'dni', 'ghi', 'albedo', 'pressure', 'wind_direction', 'wind_speed'
    ]]
    
    return df, metadata

In [ ]:
building_areas_m2 = {
    "Working Class House": 30,
    "Lower Middle Class House": 75,
    "Middle Class House": 100,
    "Upper Middle Class House": 140,
    "Upper Class House": 300
}

# roof area, assuming the whole roof is tilted at that degree

roof_area_m2 = {k: v / math.cos(ROOF_ANGLE * math.pi / 180) for k, v in building_areas_m2.items()}

In [ ]:
df, metadata = get_tmy_from_nsrdb(jakarta_lat, jakarta_lon, jakarta_tz, api_key, full_name, email, affiliation, data_source='himawari')
df.index = pd.DatetimeIndex(pd.date_range(datetime(2023, 1, 1), datetime(2024, 1, 1), inclusive='left', freq='1h', tz='UTC').tz_convert(jakarta_tz))

In [ ]:
times = pd.date_range(datetime(2023, 1, 1), datetime(2024, 1, 1), inclusive='left', freq='1h', tz='UTC').tz_convert(jakarta_tz)
solarposition_df = pvlib.location.Location(jakarta_lat, jakarta_lon).get_solarposition(times)

In [ ]:
# now calculate POA
surface_tilt = 30
surface_azimuths = [0, 90, 180, 270]
solar_zenith = solarposition_df['apparent_zenith']
solar_azimuth = solarposition_df['azimuth']
dni = df['dni']
ghi = df['ghi']
dhi = df['dhi']

# for each of the different azimuth orientations, get the poa W/m2 for each timestamp
# then, convert to kwh/h/m2 for each timestamp
# finally, convert to kwh/h for each roof slope


kwh_m2_daily_list = []
kwh_m2_monthly_list = []
for azimuth in surface_azimuths:
    total_irradiance_df = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, solar_zenith, solar_azimuth, dni, ghi, dhi)
    daily_kwh_m2 = total_irradiance_df['poa_global'].resample('1D').sum().rename(f'poa_{azimuth}') * 0.2 / 1000
    monthly_kwh_m2 = total_irradiance_df['poa_global'].resample('1MS').sum().rename(f'poa_{azimuth}') * 0.2 / 1000

    kwh_m2_list.append(daily_kwh_m2)

daily_kwh_m2_average = pd.concat(kwh_m2_list, axis=1).mean(axis=1).rename('kwh_per_m2_average')
    

In [ ]:
daily_kwh_m2_average.plot()